In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)
max_num_subprograms, max_cmd_len, max_act_len, max_actions_per_subprogram


(7, 10, 49, 9)

In [4]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap


In [5]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)
for li in commands:
    for wd in li:
        uni_commands.add(wd)
for li in actions:
    for wd in li:
        uni_actions.add(wd)
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')
num_cmd = len(uni_commands)
num_act = len(uni_actions)
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [6]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [7]:

commands_ind = [[command_map[c] for c in cmd] + [0] * (max_cmd_len - len(cmd)) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct) - 1)
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [8]:
tf.reset_default_graph()
default_sizes = 128
size_emb = 64
num_layers_encoder = 6
hidden_filters = 128
num_layers_subprogram = 3
hidden_filters_subprogram = 128
init_mag = 1e-3
cmd_mat = tf.Variable(init_mag*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(init_mag*tf.random_normal([num_act, size_emb]))
act_st_emb = tf.Variable(init_mag*tf.random_normal([size_emb]))
global_bs = None
global_time_len = 7
action_lengths = None
max_num_actions= None
# global_bs = 8
global_time_len = 7
max_num_actions = 9
output_keep_prob = tf.placeholder_with_default(1.0, ())
state_keep_prob = tf.placeholder_with_default(1.0, ())
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, 10,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs,))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))
learning_rate = tf.placeholder(tf.float32, shape = (None))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)
tf_bs = tf.shape(act_ind)[0]
act_st_emb_expanded = tf.tile(tf.reshape(
    act_st_emb, [1, 1, 1, size_emb]), [tf_bs, global_time_len, 1, 1])
act_emb_with_st = tf.concat((act_st_emb_expanded, act_emb), 2)

first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
hidden_cells_encoder = [[tf.nn.rnn_cell.DropoutWrapper(cell,
    output_keep_prob=output_keep_prob, state_keep_prob=state_keep_prob,
    variational_recurrent=True, dtype=tf.float32) for cell in cells] 
                        for cells in hidden_cells_encoder[:-1]] + [hidden_cells_encoder[-1]]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf_bs
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        if idx == num_layers - 1:
            fw = prev_layer[0]
            bw = prev_layer[1]
            stacked = tf.stack([tf.range(bs), lengths - 1], 1)
            fw_final = tf.gather_nd(fw,stacked,name=None)
            bw_final = bw[:,0,:]
            output = tf.concat((fw_final, bw_final), 1)
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            #stacked = tf.stack([tf.range(bs), lengths - 1], 1)
            #output = tf.gather_nd(prev_layer,stacked,name=None)
            return prev_layer, returncells, hiddenlayers, output, fw, stacked
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep, dbg1, dbg2 = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]

cells_subprogram_rnn = [first_cell_subprogram] + hidden_cells_subprogram

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    num_units=hidden_filters_encoder, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=hidden_filters_encoder//2, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
cells_subprogram = [
    tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanism, attention_layer_size = hidden_filters_subprogram) 
    for cell in cells_subprogram_rnn]

def subprogram(x, num_layers, cells, initial_states, lengths, reuse, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        print(idx)
        if idx == 0:
            num_past_units = hidden_filters
        else:
            num_past_units = hidden_filters_subprogram
        with tf.variable_scope(name + 'subprogram' + str(idx), reuse=reuse):
#             self_attention_mechanism = tf.contrib.seq2seq.LuongAttention(
#                 num_units=num_past_units, memory=prev_layer,
#                 memory_sequence_length=tf.expand_dims(tf.range(10), 0))
#             cell_with_selfattention = tf.contrib.seq2seq.AttentionWrapper(
#                     cells[idx], self_attention_mechanism, attention_layer_size = num_past_units)

            prev_layer, c = tf.nn.dynamic_rnn(
                    cell = cells[idx],
                    inputs = prev_layer,
                    sequence_length=lengths,
                    initial_state = None,
                    dtype=tf.float32,
                )
            prev_layer = tf.concat(prev_layer, 2)
            prev_layer = tf.nn.leaky_relu(prev_layer)
            returncells.append(c)
            hiddenlayers.append(prev_layer)
            if idx == num_layers - 1:
                output = tf.gather_nd(
                            prev_layer,
                            tf.stack([tf.range(bs), lengths], 1),
                            name=None
                        )
                return prev_layer, returncells, hiddenlayers, output
            prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms): 
    from_last_layer = tf.tile(tf.expand_dims(tf.concat((
        initial_cmb_encoding, last_encoding), 1), 1), [1, max_num_actions + 1, 1])
    autoregressive = act_emb_with_st[:,sub_idx, :, :]
    x_input = tf.concat((from_last_layer, autoregressive), -1)
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        x_input, num_layers_subprogram, cells_subprogram,None, 
        lengths = act_lengths[:, sub_idx], reuse = (sub_idx > 0), name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions + 1, num_act])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta1, delta2 = [mlp(
        subprogram_output, [256,], output_size = hidden_filters_encoder, name = 'global_transform' + str(idx),
        reuse = (sub_idx > 0)
    ) for idx in range(2)]
    remember = tf.sigmoid(delta1)
    insert = tf.tanh(delta2) + delta2/100
    last_encoding = last_encoding * remember + insert
    encodings.append(last_encoding)
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
ppl_loss_avg = tf.reduce_mean(tf.pow(ppl_loss, 2.0)) * 10000 # + tf.reduce_mean(tf.pow(ppl_loss, 1.0)) * 100

tfvars = tf.trainable_variables()
weight_norm = tf.reduce_mean([tf.reduce_sum(tf.square(var)) for var in tfvars])*1e-3

action_taken = tf.argmax(act_presoftmax, -1, output_type=tf.int32)
correct_mat = tf.cast(tf.equal(action_taken, act_ind), tf.float32) * mask_ph
correct_percent = tf.reduce_sum(correct_mat, [1, 2])/tf.reduce_sum(mask_ph, [1, 2])
percent_correct = tf.reduce_mean(correct_percent)
percent_fully_correct = tf.reduce_mean(tf.cast(tf.equal(correct_percent, 1), tf.float32))

loss = ppl_loss_avg + weight_norm

opt_fcn = tf.train.AdamOptimizer(learning_rate=learning_rate)
#opt_fcn = tf.train.MomentumOptimizer(learning_rate=learning_rate, use_nesterov=True, momentum=.8)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
hidden_filters

128

In [10]:
encoding_last_layer

<tf.Tensor 'LeakyRelu_5/Maximum:0' shape=(?, 10, 256) dtype=float32>

In [ ]:
np.random.seed(0)
trn_percent = .1
num_samples = mask_np.shape[0]
ordered_samples = np.arange(num_samples)
np.random.shuffle(ordered_samples)
trn_samples = ordered_samples[:int(np.ceil(num_samples*trn_percent))]
val_samples_all = ordered_samples[int(np.ceil(num_samples*trn_percent)):]
val_samples = val_samples_all
trn_samples.shape, val_samples.shape

((2091,), (18819,))

In [ ]:
eval_itr = -1
bs = 32# trn_samples.shape[0]
for itr in range(1000000):
    if 1:#itr == 0:
        samples = np.random.choice(trn_samples, size = bs, replace = False)
        trn_feed_dict = {
            cmd_ind : cmd_np[samples],
            act_ind : act_np[samples],
            mask_ph : mask_np[samples],
            act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[samples],
        }
        
    trn_feed_dict[learning_rate] = .02 / (np.power(itr + 10, .6))
    _, trn_loss, acc_trn_single, acc_trn = sess.run(
        [optimizer, loss, percent_correct, percent_fully_correct], trn_feed_dict)
    if itr == 0:
        trn_loss_avg = trn_loss
        acc_trn_avg = acc_trn
        acc_trn_single_avg = acc_trn_single
    else:
        trn_loss_avg = trn_loss_avg * .9 + trn_loss * .1
        acc_trn_avg = acc_trn_avg * .9 + acc_trn * .1
        acc_trn_single_avg = acc_trn_single_avg * .9 + acc_trn_single * .1
    if itr % 10 == 0 and itr > 0:
        # val_samples = np.random.choice(val_samples_all, size = bs, replace = False)
        eval_itr += 1
        val_feed_dict = {
            cmd_ind : cmd_np[val_samples],
            act_ind : act_np[val_samples],
            mask_ph : mask_np[val_samples],
            act_lengths : np.clip(struct_np[val_samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[val_samples]
        }
        val_loss, acc_val = sess.run([loss, percent_fully_correct], val_feed_dict)
        if eval_itr == 0:
            val_loss_avg = val_loss
            acc_val_avg = acc_val
        else:
            val_loss_avg = val_loss_avg * .9 + val_loss * .1
            acc_val_avg = acc_val_avg * .9 + acc_val * .1
        print('itr:', itr, 'trn_loss', trn_loss_avg, 'val_loss', val_loss_avg)
        print('itr:', itr, 'trn_acc', acc_trn_avg, 
              'trn_single_acc', acc_trn_single_avg, 'val_acc', acc_val_avg)

itr: 10 trn_loss 12259.533095199638 val_loss 11022.221
itr: 10 trn_acc 0.0 trn_single_acc 0.20621468693861753 val_acc 0.0
itr: 20 trn_loss 10818.999364477964 val_loss 10904.6412109375
itr: 20 trn_acc 0.0 trn_single_acc 0.30905821685488666 val_acc 0.0
itr: 30 trn_loss 9955.256869959623 val_loss 10761.01322265625
itr: 30 trn_acc 0.0 trn_single_acc 0.35531785080658684 val_acc 2.1255113824736328e-05
itr: 40 trn_loss 9279.634394687717 val_loss 10581.255455078126
itr: 40 trn_acc 0.0 trn_single_acc 0.4007436931342362 val_acc 1.9129602442262695e-05
itr: 50 trn_loss 8019.355160170751 val_loss 10289.400808007815
itr: 50 trn_acc 0.0 trn_single_acc 0.44724740464246493 val_acc 2.2530420654220508e-05
itr: 60 trn_loss 7246.081843264035 val_loss 9887.434164707034
itr: 60 trn_acc 0.0022781250000000006 trn_single_acc 0.4779560537893881 val_acc 2.0277378588798456e-05
itr: 70 trn_loss 6303.4921487948895 val_loss 9441.278980658206
itr: 70 trn_acc 0.0007943330713528129 trn_single_acc 0.5022768449926371 val_

itr: 520 trn_loss 1136.1068411851136 val_loss 1569.2973483571843
itr: 520 trn_acc 0.02930310023190616 trn_single_acc 0.814447227554443 val_acc 0.014766266975377178
itr: 530 trn_loss 1091.397790833862 val_loss 1537.9024401816223
itr: 530 trn_acc 0.038289197435205 trn_single_acc 0.8174169907762406 val_acc 0.015377955317712584
itr: 540 trn_loss 1032.174665399781 val_loss 1504.7058363001788
itr: 540 trn_acc 0.03796234363438822 trn_single_acc 0.8256690960732065 val_acc 0.016236673866406166
itr: 550 trn_loss 1069.7715568374592 val_loss 1472.8693957365672
itr: 550 trn_acc 0.031165595510978653 trn_single_acc 0.8211349539190393 val_acc 0.01715830641957678
itr: 560 trn_loss 1099.3452624339955 val_loss 1450.866513780098
itr: 560 trn_acc 0.031670856818180604 trn_single_acc 0.8108233004838867 val_acc 0.017578614790492275
itr: 570 trn_loss 1100.503280043516 val_loss 1426.5535806638072
itr: 570 trn_acc 0.03674578119886869 trn_single_acc 0.8133719110427869 val_acc 0.01820132612966271
itr: 580 trn_loss

itr: 1030 trn_loss 562.1169340575229 val_loss 876.2139722881017
itr: 1030 trn_acc 0.12365536963165268 trn_single_acc 0.8846345177144058 val_acc 0.0689186599505488
itr: 1040 trn_loss 578.7566929335607 val_loss 875.7222747663228
itr: 1040 trn_acc 0.12158703363377855 trn_single_acc 0.8894846246686849 val_acc 0.06939700513951874
itr: 1050 trn_loss 595.2932196468706 val_loss 879.2514938228936
itr: 1050 trn_acc 0.11177807603006215 trn_single_acc 0.884170940613855 val_acc 0.07004006671791878
itr: 1060 trn_loss 589.3926030416401 val_loss 874.4111222726356
itr: 1060 trn_acc 0.1338775807687913 trn_single_acc 0.8872211103135885 val_acc 0.07179848076002841
itr: 1070 trn_loss 592.5589435530659 val_loss 868.9559170277939
itr: 1070 trn_acc 0.13266321503932393 trn_single_acc 0.8879237559531117 val_acc 0.07238206316021423
itr: 1080 trn_loss 552.1941924680328 val_loss 861.0403668289208
itr: 1080 trn_acc 0.14973219065043733 trn_single_acc 0.8937499349182174 val_acc 0.07396472959805138
itr: 1090 trn_loss 

itr: 1540 trn_loss 386.87594611432115 val_loss 738.1160731169015
itr: 1540 trn_acc 0.2368940607205116 trn_single_acc 0.9142306323652387 val_acc 0.10495087681648618
itr: 1550 trn_loss 397.5878618447133 val_loss 736.0748759614613
itr: 1550 trn_acc 0.20111244945785775 trn_single_acc 0.9125797676466835 val_acc 0.10506209145579476
itr: 1560 trn_loss 404.46702775498744 val_loss 731.0790338731276
itr: 1560 trn_acc 0.23140519632103096 trn_single_acc 0.9125870051915563 val_acc 0.10499745750958091
itr: 1570 trn_loss 392.79656264689436 val_loss 729.1496888354243
itr: 1570 trn_acc 0.20296065969675137 trn_single_acc 0.9140684062800323 val_acc 0.10526874120117156
itr: 1580 trn_loss 373.91564356168874 val_loss 724.913021953835
itr: 1580 trn_acc 0.24468274043098032 trn_single_acc 0.9162610321341967 val_acc 0.10718673650992747
itr: 1590 trn_loss 355.60020691176493 val_loss 723.8850071119672
itr: 1590 trn_acc 0.25968741977161747 trn_single_acc 0.9180524352658744 val_acc 0.10776515619074853
itr: 1600 trn

itr: 2050 trn_loss 299.320347970503 val_loss 754.7173211608101
itr: 2050 trn_acc 0.2933105577489195 trn_single_acc 0.9299214319946095 val_acc 0.12944732050397115
itr: 2060 trn_loss 288.99481060421635 val_loss 754.5865558416042
itr: 2060 trn_acc 0.3059410404595044 trn_single_acc 0.9308269974737298 val_acc 0.1300686652090977
itr: 2070 trn_loss 293.4743655988509 val_loss 756.988752308225
itr: 2070 trn_acc 0.31944372006874094 trn_single_acc 0.9318720927251113 val_acc 0.1316321794355062
itr: 2080 trn_loss 299.0103312733674 val_loss 760.8442166770119
itr: 2080 trn_acc 0.3240077419039348 trn_single_acc 0.9320713822185294 val_acc 0.13231666870392686
itr: 2090 trn_loss 319.6613293280023 val_loss 766.3465564839202
itr: 2090 trn_acc 0.30260284162113743 trn_single_acc 0.9282478094301411 val_acc 0.13278392244110102
itr: 2100 trn_loss 303.7480174832198 val_loss 765.7369679742
itr: 2100 trn_acc 0.3116227621050357 trn_single_acc 0.9339216057674651 val_acc 0.13342231621202463
itr: 2110 trn_loss 290.690

...

itr: 39180 trn_loss 0.04205634344648089 val_loss 15.550211103155554
itr: 39180 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9900381852375884
itr: 39190 trn_loss 0.042054461697543274 val_loss 15.549838678295322
itr: 39190 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9900406917203145
itr: 39200 trn_loss 0.042060032198511545 val_loss 15.495238187540984
itr: 39200 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9900535750629316
itr: 39210 trn_loss 0.04203999029906247 val_loss 15.464412107434347
itr: 39210 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9900704808451364
itr: 39220 trn_loss 0.042019438243145614 val_loss 15.453473807304926
itr: 39220 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9900803852752712
itr: 39230 trn_loss 0.04199876885771546 val_loss 15.395506549712861
itr: 39230 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.990094610036

itr: 39680 trn_loss 0.0413680694651468 val_loss 16.547239208306586
itr: 39680 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9896858775799497
itr: 39690 trn_loss 0.04135053521605467 val_loss 16.634022245483738
itr: 39690 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.989681104795786
itr: 39700 trn_loss 0.04133913591777793 val_loss 16.68891931445343
itr: 39700 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9896555542737118
itr: 39710 trn_loss 0.04134202197859883 val_loss 16.751561005749785
itr: 39710 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9896325588038448
itr: 39720 trn_loss 0.041318980092680244 val_loss 16.836074018333985
itr: 39720 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9896278011629775
itr: 39730 trn_loss 0.04130656372653992 val_loss 16.882418475021094
itr: 39730 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9896182085123475


itr: 40180 trn_loss 0.04053046443509883 val_loss 17.162119973643147
itr: 40180 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9892902423093952
itr: 40190 trn_loss 0.040500193026173394 val_loss 17.1729209920259
itr: 40190 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9892612680033062
itr: 40200 trn_loss 0.04048866469974896 val_loss 17.2351855487071
itr: 40200 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9892245636196626
itr: 40210 trn_loss 0.040461016249366884 val_loss 17.25579598400973
itr: 40210 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9892180954645597
itr: 40220 trn_loss 0.04043700530552742 val_loss 17.236524559742058
itr: 40220 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9892335291412941
itr: 40230 trn_loss 0.04042365808862362 val_loss 17.208668498115998
itr: 40230 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.989252736184669
it

itr: 40680 trn_loss 0.0394898482069323 val_loss 18.144660502887394
itr: 40680 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9890176444149665
itr: 40690 trn_loss 0.03945412062825717 val_loss 18.241872576072776
itr: 40690 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9890212406721698
itr: 40700 trn_loss 0.039437449082930724 val_loss 18.46554737138298
itr: 40700 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9889766564971495
itr: 40710 trn_loss 0.03940902011661853 val_loss 18.717196102567435
itr: 40710 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9889099589889904
itr: 40720 trn_loss 0.039393547869216054 val_loss 20.85466419067495
itr: 40720 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9886958293830448
itr: 40730 trn_loss 0.039408485529935984 val_loss 22.340532229004918
itr: 40730 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.988301190082588

itr: 41180 trn_loss 0.039500700365076934 val_loss 13.775825469308897
itr: 41180 trn_acc 0.999999998773119 trn_single_acc 0.9999999999233198 val_acc 0.9907059562504041
itr: 41190 trn_loss 0.0395056271974347 val_loss 13.361611147475664
itr: 41190 trn_acc 0.999999999572213 trn_single_acc 0.9999999999732633 val_acc 0.990790470746137
itr: 41200 trn_loss 0.0395098643764857 val_loss 12.970950597303291
itr: 41200 trn_acc 0.9999999998508399 trn_single_acc 0.9999999999906775 val_acc 0.9908771613004601
itr: 41210 trn_loss 0.03949245889864386 val_loss 12.617129463354212
itr: 41210 trn_acc 0.999999999947991 trn_single_acc 0.9999999999967494 val_acc 0.9909764378156778
itr: 41220 trn_loss 0.039480105587740316 val_loss 12.328738527059073
itr: 41220 trn_acc 0.9999999999818655 trn_single_acc 0.9999999999988665 val_acc 0.9910764141875372
itr: 41230 trn_loss 0.03945913088344548 val_loss 12.114202400946184
itr: 41230 trn_acc 0.9999999999936768 trn_single_acc 0.9999999999996048 val_acc 0.9911504486797332
it

In [ ]:
correct_percent.shape, percent_fully_correct.shape

In [ ]:
10, None, 7, 9, None

In [ ]:
cmd_np.shape, act_np.shape, mask_np.shape, struct_np.shape, cmd_lengths_np.shape

In [ ]:
cmd_ind.shape, act_ind.shape, mask_ph.shape, act_lengths.shape, cmd_lengths.shape

In [ ]:
trn_samples.shape, val_samples.shape

In [ ]:

act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])

In [ ]:
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])

In [ ]:
max_actions_per_subprogram

In [ ]:
sess.run(act_presoftmax, feed_dict).shape

In [ ]:
action_map

In [ ]:
print(*actions_ind[2])

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape